In [2]:
import json
import math
import os
import numpy as np
import pandas as pd
import requests
import browser_cookie3
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
%matplotlib inline

In [3]:
if not os.path.exists('input15'):
    cookies = browser_cookie3.firefox(domain_name="adventofcode.com")
    session_cookie = [c for c in list(cookies) if c.name == "session"][0].value
    resp = requests.get('https://adventofcode.com/2021/day/15/input', cookies={'session': session_cookie})
    with open('input15', 'w') as f:
        f.write(resp.text)

In [4]:
!head input15

7135912411912798932871391322889941544645211112288183969191588665579426181549613954113914616349281119
8818482819182139319112316373697999126211219541956811442497469891197212131227119531231231297911937841
1514575112959236914131559711156719336447259942553728322911271774241293394881913682176891871931225931
3911411211611179328267522115348239963876196416413136271519591723261183181259792938429689524986199662
7641142151561121642912253491632132312682391976219236246811321441614458168298442897517119286294427143
2121192132289162313728563181871122493359816197929334844219181379799666376121879896792359861963352341
7824634877975124922137296299957468954179392824124437392871314337752916614225212219614414683959191151
4697413917922193912419678321922133939613861216689158123571322738228138479426524278182172119312848883
8924576358796711856729358848547891622733114113537182199512138345191419912781799661213494977849752719
9132169172714432689989487585816291188372945729446982887385921614921294423815188486235978437

In [123]:
with open('input15', 'r') as f:
    lines = [l.strip() for l in f.readlines()]

## Part 1

In [6]:
lines = [
    '1163751742',
    '1381373672',
    '2136511328',
    '3694931569',
    '7463417111',
    '1319128137',
    '1359912421',
    '3125421639',
    '1293138521',
    '2311944581',
]

In [120]:
height = len(lines)
width = len(lines[0])
costs = np.zeros((height, width))
for i, line in enumerate(lines):
    costs[i,:] = [int(v) for v in line]

In [17]:
Q = set()
dist = {}
prev = {}
for r in range(height):
    for c in range(width):
        dist[(r, c)] = 1e100
        prev[(r, c)] = None
        Q.add((r, c))
dist[(0, 0)] = 0

while len(Q) > 0:
    min_dist = 1e200
    min_q = None
    for q in Q:
        if dist[q] < min_dist:
            min_dist = dist[q]
            min_q = q
    u = min_q
    Q.remove(u)
    ur, uc = u
    if ur > 0:
        v = (ur - 1, uc)
        alt = dist[u] + costs[v]
        if alt < dist[v]:
            dist[v] = alt
            prev[v] = u
    if ur < width - 1:
        v = (ur + 1, uc)
        alt = dist[u] + costs[v]
        if alt < dist[v]:
            dist[v] = alt
            prev[v] = u
    if uc > 0:
        v = (ur, uc - 1)
        alt = dist[u] + costs[v]
        if alt < dist[v]:
            dist[v] = alt
            prev[v] = u
    if uc < height - 1:
        v = (ur, uc + 1)
        alt = dist[u] + costs[v]
        if alt < dist[v]:
            dist[v] = alt
            prev[v] = u

In [18]:
dist[(height-1, width-1)]

441.0

## Part 2

In [124]:
orig_height = len(lines)
orig_width = len(lines[0])
orig_costs = np.zeros((orig_height, orig_width))
for i, line in enumerate(lines):
    orig_costs[i,:] = [int(v) for v in line]

In [125]:
height = orig_height * 5
width = orig_width * 5
costs = np.zeros((height, width))
for i in range(5):
    for j in range(5):
        costs[i*orig_height:(i+1)*orig_height, j*orig_width:(j+1)*orig_width] = (orig_costs - 1) + i + j
costs = (costs % 9) + 1

In [126]:
def get_h_cost(u, v):
    ux, uy = u
    vx, vy = v
    return abs(ux - vx) + abs(vx - vy)

In [127]:
Q = set()
closed = set()
dist = {}
fdist = {}
prev = {}
Q.add((0, 0))
dist[(0, 0)] = 0
fdist[(0, 0)] = 0 + get_h_cost((0, 0), (height-1, width-1))

while len(Q) > 0:
    min_dist = 1e200
    min_q = None
    for q in Q:
        if fdist[q] < min_dist:
            min_dist = fdist[q]
            min_q = q
    u = min_q
    if u == (height-1, width-1):
        break
    Q.remove(u)
    closed.add(u)
    ur, uc = u
    neighbours = []
    if ur > 0:
        neighbours.append((ur - 1, uc))
    if ur < width - 1:
        neighbours.append((ur + 1, uc))
    if uc > 0:
        neighbours.append((ur, uc - 1))
    if uc < height - 1:
        neighbours.append((ur, uc + 1))
    for v in neighbours:
        if v in closed:
            continue
        alt = dist[u] + costs[v]
        if v not in Q:
            Q.add(v)
            prev[v] = u
            dist[v] = alt
            fdist[v] = alt + get_h_cost(u, v)
        elif alt < dist[v]:
            prev[v] = u
            dist[v] = alt
            fdist[v] = alt + get_h_cost(u, v)

In [128]:
dist[(height-1, width-1)]

2849.0